In [66]:
import numpy as np
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string,preprocess_documents
from gensim import corpora,models, similarities
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import log_loss
from sklearn.cross_validation import cross_val_score, train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
import re
import xgboost as xgb
color = sns.color_palette()
%matplotlib inline
from xgboost import XGBClassifier
from sklearn import metrics

In [46]:
df1 = pd.DataFrame.from_csv('training_set_rel3.tsv', sep='\t')

In [47]:
df1.head

<bound method DataFrame.head of           essay_set                                              essay  \
essay_id                                                                 
1                 1  Dear local newspaper, I think effects computer...   
2                 1  Dear @CAPS1 @CAPS2, I believe that using compu...   
3                 1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
4                 1  Dear Local Newspaper, @CAPS1 I have found that...   
5                 1  Dear @LOCATION1, I know having computers has a...   
6                 1  Dear @LOCATION1, I think that computers have a...   
7                 1  Did you know that more and more people these d...   
8                 1  @PERCENT1 of people agree that computers make ...   
9                 1  Dear reader, @ORGANIZATION1 has had a dramatic...   
10                1  In the @LOCATION1 we have the technology of a ...   
11                1  Dear @LOCATION1, @CAPS1 people acknowledge the...   
12    

In [48]:
df1.isnull().sum()

essay_set             0
essay                 0
rater1_domain1        0
rater2_domain1        0
rater3_domain1    12848
domain1_score         0
rater1_domain2    11176
rater2_domain2    11176
domain2_score     11176
rater1_trait1     10684
rater1_trait2     10684
rater1_trait3     10684
rater1_trait4     10684
rater1_trait5     12253
rater1_trait6     12253
rater2_trait1     10684
rater2_trait2     10684
rater2_trait3     10684
rater2_trait4     10684
rater2_trait5     12253
rater2_trait6     12253
rater3_trait1     12848
rater3_trait2     12848
rater3_trait3     12848
rater3_trait4     12848
rater3_trait5     12848
rater3_trait6     12848
dtype: int64

In [49]:
#dropping the other scores because those are being evaluated as a different metrics 
#and due to time constraints we don't have time to work with those yet
df1.dropna(axis =1, inplace=True)

In [50]:
df1.head()

,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score
essay_id,,,,,
1,1,"Dear local newspaper, I think effects computer...",4,4,8
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10
5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8


In [51]:
df1.to_csv('newdf.csv')

In [53]:
df1 = pd.read_csv('newdf.csv', encoding="ISO-8859-1")

In [63]:
#basic models with simple count vectorizing with no additional features to predict individual and domain scores

c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
c_vect.fit(df1['essay'])

c_vect_sparse_1 = c_vect.transform(df1['essay'])
c_vect_sparse1_cols = c_vect.get_feature_names()

pred_feats = ['essay_set']

df2 = df1['essay_set']
df2 = pd.DataFrame(df2)

df_cv1_sparse = sparse.hstack((df2.astype(float), c_vect_sparse_1)).tocsr()
pred_feats += c_vect_sparse1_cols

y_train1 = df1.rater1_domain1
y_train2 = df1.rater2_domain1
y_train3 = df1.domain1_score




In [67]:
df_cv1_sparse.shape

(12976, 201)

In [69]:
rf = RandomForestClassifier(n_estimators=1000, max_depth= 17, min_samples_leaf= 1, oob_score=True)
rf.fit(df_cv1_sparse, y_train1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=17, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [72]:
predicted = cross_val_predict(rf, df_cv1_sparse, y_train1, cv=5)


NameError: name 'metrics' is not defined

In [74]:
metrics.accuracy_score(y_train1, predicted)

0.39773427866831074

In [78]:
#note GBM in sklearn only accepts dense arrays during prediction step
gbm0 = GradientBoostingClassifier(n_estimators=2000, max_features=5, subsample= 0.8)
gbm0.fit(df_cv1_sparse.toarray(), y_train1)

predicted1 = cross_val_predict(gbm0, df_cv1_sparse.toarray(), y_train1, cv=5)
metrics.accuracy_score(y_train1, predicted1)

0.42678791615289768

In [79]:
from sklearn.externals import joblib
joblib.dump(gbm0, 'gbm_model0.pkl')
joblib.dump(rf, 'rf_model0.pkl')

['rf_model0.pkl']

In [ ]:
#use text from sample questions to create query topics
#extract text from essay docs, cosine similarity?

In [80]:
#acquiring corpus from both some gutenberg text as well as those from a "big" text file downloaded from here:
# https://github.com/luistp001/LT-Autograder/tree/master/AdditionalFiles
from gutenberg.cleanup import strip_headers
import requests
# http://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

def load_etext(num):
  url ='http://www.gutenberg.org/files/{0}/{0}-0.txt'.format(num)
  data= requests.get(url)
  return data.text

texts =[]
text_nums=[1342, 158, 121, 105, 141,
           76,1837, 86, 3176, 8525,
           11,12, 29042, 620,35535,
           2701,21816, 34970, 12384,2694,
           98,19337, 786, 1023, 1400,
           2591]
text_names =['Pride and Prejudice', 'Emma', 'Northanger Abbey', "Persuasion", "Mansfield Park",
             'Adventures of Huckleberry Finn', 'The Prince and the Pauper', "A Connecticut Yankee in King Arthur's Court", "The Innocents Abroad", "Eve's Diary", 
             "Alice in Wonderland","Through the Looking-Glass", "A Tangled Tale", "Sylvie and Bruno", "Feeding the Mind",
            "Moby Dick", "The Confidence-Man: His Masquerade", "Pierre; or The Ambiguities", "Battle-Pieces and Aspects of the War","I and My Chimney",
             "A Tale of Two Cities","A Christmas Carol","Hard Times", "Bleak House", "Great Expectations",
             "Grimms’ Fairy Tales", 
            ]
for text_num in text_nums:
    text = strip_headers(load_etext(text_num)).strip()
    texts.append(text)


In [81]:
processed_docs = preprocess_documents(texts)
dictionary = corpora.Dictionary(processed_docs)
dictionary.save('gutenberg.dict')
corpus = [dictionary.doc2bow(text) for text in processed_docs]
corpora.MmCorpus.serialize('gutenberg.mm', corpus) 

In [82]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=500)
lsi.save("gutenberg.model")

In [83]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=50, iterations= 500)
lda.save('ldagutenberg.model')

In [ ]:
index = similarities.MatrixSimilarity(lsi[corpus])
# similarities.Similarity
index.save('gutenberg.index')

In [ ]:
indexLda = similarities.MatrixSimilarity(lda[corpus])
indexLda.save('ldagutenberg.index')

In [103]:
bigcorpus = corpora.TextCorpus(input = 'big.txt')

In [104]:
bigcorpus

TypeError: 'Dictionary' object is not callable

In [98]:
from gensim.models import Doc2Vec
# [Doc2Vec Parameters]
vec_dim = 30
win_size = 1
alpha = 0.01
min_count = 2
sample = 1e-5
workers = 100
epochs = 10
# build
model = Doc2Vec(size=int(vec_dim), window=win_size, alpha=alpha, min_count=min_count, sample=sample, workers=workers)
model.build_vocab(dictionary)
tmpList = dictionary
print 'Training Model...'
for e in range(epochs):
    print '\tEpoch ' + str(e)
random.shuffle(tmpList)
model.alpha -= 0.001
model.min_alpha = model.alpha


AttributeError: 'int' object has no attribute 'words'